# Архитектура трансформеров

Архитектура трансформеров примечательна своей эффективностью в обработке последовательностей данных, таких как предложения или изображения. Трансформеры представляют собой нейросетевую архитектуру, которая отличается от рекуррентных нейронных сетей (RNN) и обеспечивает высокую параллелизацию. Они используют механизм внимания для создания векторного представления каждого элемента последовательности, что позволяет эффективно обрабатывать контекст каждого элемента. Эта архитектура нашла широкое применение не только в машинном переводе, но и в обработке изображений, где она позволяет улучшить скорость обучения, точность и размер выборки.

### Энкодеры и декодеры



Механизм внимания связывает разные элементы последовательности для вычисления представления этой последовательности. Его архитектура состоит из энкодеров, декодеров и связи между ними.

Энкодер отображает входную последовательность символьных представлений $(x_1, ..., x_n)$ в последовательность внутренних представлений $z = (z_1, ..., z_n)$. На основе $z$ декодер генерирует выходную последовательность $(y_1, ..., y_n)$ символов по одному элементу за единицу времени. 

![stack](images/transformer_stack.png)

Энкодер состоит из стека 6 одинаковых слоев. У каждого слоя есть 2 под-слоя: слоя многоголового внутреннего внимания и позиционного слоя полносвязной сети прямого распространения. Каждый под-слой обернут остаточной связью (residual connection) и слоем нормализации. Чтобы облегчить работу с остаточными связями, все под-слои в модели, так же как и слой эмбеддингов, производят выходные данные размерности  $d_{model}$.  За счет такого строения каждого слоя энкодер может видеть весь контекст вокруг слова (справа и слева).

![encoder-decoder](images/transformer_encoder.png)

Декодер также представляет собой стек из 6 одинаковых слоев. Помимо двух под-слоев, как в энкодере, декодер содержит дополнительный под-слой, который применяет многоголовое внутреннее внимание на выходной вектор энкодера. Слой внимания здесь модифицирован: чтобы модель не использовала контекст справа отрассматриваемого токена, а опиралась только на известную (левую) часть последовательности, добавляются маски.

![encoder-decoder](images/transformer_encoder_decoder.png)

### Функция внимания



![attrention](images/transformer_attention.png)

Функция внимания представляет собой отображение вектора запроса и набора пар векторов ключ-значение на выходное значение. Выходное значение вычисляется как взвешенная сумма значений, где вес присваивается каждому значению через вычисление соответствия запроса и ключа. 

Вход состоит из запросов, ключей размерности $d_k$ и значений размерности $d_v$. Мы вычисляем скалярное произведение запроса со всеми ключами, делим каждое на $\sqrt {d_k}$ и затем считаем $softmax$ для получения весов значений. На практике веса считаются сразу для матрицы запросов $Q$. Ключи и значения также объединяются в матрицы $K$ и $V$:

$Attention(Q, K, V) = softmax(\frac {QK^T} {\sqrt {d_k}})V$

Нормировка значений весов с помощью $\frac 1 {\sqrt {d_k}}$ делается для борьбы с проблемой затухающих градиентов. Они могут появиться при больших значениях $d_k$. Скалярное произведение попадает в функцию $softmax$ и отображается на область с экстремально низкими значениями градиента.

На практике оказалось более успешным использовать $h$ обученных линейных проекций запросов, ключей и значений в измерения $d_q$, $d_k$ и $d_v$ соответственно. Для каждой из полученных проекций параллельно вычисляется функция внимания. Полученные векторы размерности $d_v$ конкатенируются и подаются в линейный слой для получения финальных весов. 

$MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O$ where $head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)$  

![stack](images/transformer_architecture.png)

Многоголовое внутреннее внимание используется тремя разными способами:
- Энкодер-декодер: запросы приходят с предыдущего слоя декодера, а ключи и значения - с предыдущего слоя энкодера. Это позволяет каждой позиции в декодере строить связи со всеми позициями во входящей последовательности;
- Слои энкодера: ключи, значения и запросы поступают из одного и того же источника - предыдущего слоя энкодера. Каждая позиция в энкодере связана со всеми позициями в предыдущем слое энкодера;
- Слои декодера: позволяют каждой позиции декодера быть связанными со всеми позициями предыдущего слоя декодера ДО текущей позиции (левый контекст). Декодер не должен видеть информацию справа, и это реализуется как замена всех недопустимых связей на $-\infty$ перед подачей в $softmax$.

### Feed Forward Layer


Каждый слой энкодера и декодера содержит полносвязную сеть прямого распространения, которая одинаково применяется ко всем позициям по-отдельности. Она состоит из двух слоев линейных преобразований с активационной функцией $ReLU$ между ними. 

$FFN(x) = max(0, xW_1 + b_1)W_2 + b_2$

Размерность входа и выхода сети прямого распространения равна $d_{model} = 512$, а размерность внутреннего слоя - $d_{ff} = 2048$.

### Позицинное кодирование

Чтобы учитывать порядок слов, требуется внедрять информацию об относительной или абсолютной позиции токена в последовательности. Для этого внизу энкодера и декодера к входным эмбеддингам добавляются позиционные эмбеддинги. В работе *Attention is all you need* используются функции синуса и косинуса различных частот:

$PE(pos, 2i) = sin(\frac {pos} {10000^{\frac {2i} {d_{model}}}})$

$PE(pos, 2i + 1) = cos(\frac {pos} {10000^{\frac {2i} {d_{model}}}})$, 

где $pos$ - это позиция, и $i$ - измерение. Каждому измерению позиционного кодирования соответствует синусоида. Эта функция была выбрана исходя из гипотезы, что это позволит модели быстрее обучаться связям между близкими позициями. Так как смещение $k$ фиксировано, то $PE_{pos+k}$ моет быть представлено в виде линейного преобразования $PE_k$. Также синусоида позволяет модели работать с последовательностями длиннее, чем в обучающей выборке, за счет экстраполяции.

Чем меньше расстояние между любыми комбинациями последовательностей на входе и на выходе сети, тем легче обрабатывать зависимости в длинном контексте. Внутреннее внимание соединяет все позиции постоянным числом последовательно выполняемых операций, где реккурентный слой требует $O(n)$ последовательных операций. В терминах вычислительной сложности слои внутреннего внимания быстрее, чем реккурентные, когда последовательность размерности $n$ меньше, чем размерность представления $d$.

### Преимущества трансформеров


- общая вычислительная сложность на слой 
- легкость распараллеливания
- расстояние между дальними зависимостями в сети

### Заключение


Внимание в нейросетях — очень влиятельная идея. Сейчас все ключевые архитектуры в обработке языка, начиная с GPT-2 от OpenAI и BERT до самых современных, полагаются на механизм внимания. Внимание можно применять не только в обработке текста: если применить механизм к сверточным нейросетям, можно визуализировать, на какие части картинки смотрела нейросеть, когда решила, что на ней изобразили собаку.